In [1]:
import pandas as pd

# add ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
myTeamName = "Team 7"

In [3]:
numTeams = 8
numPlayersPerTeam = 13
ROTO8_DRAFT_ORDER = ["Team 1", "Team 2", "Team 3", "Team 4", "Team 5", "Team 6", "Team 7", "Team 8"]
SUMMARY_STATS = ["PTS", "REB", "AST", "ST", "BLK", "TO", "3PTM"]
AVERAGE_STATS = ["FG%", "FT%"]

In [4]:
dataDir                                 =   "../data/"   
playerProjStatsFilepath                   =   dataDir + "proj_24_25_150_players.csv"

In [5]:
stats_categories                        =   SUMMARY_STATS   +   AVERAGE_STATS
relevantColumns                         =   ["Player"]      +   stats_categories
draft_columns                           =   ["Team"]        +   relevantColumns

In [6]:
def loadPlayerProjStats(playerProjStatsFilepath):       
    print("loadPlayerProjStats")
    print("Loading player stats from: ", playerProjStatsFilepath)

    dfPlayerProjStats                   =   pd.read_csv(playerProjStatsFilepath)
    dfPlayerProjStats                   =   dfPlayerProjStats[relevantColumns]
    dfPlayerProjStats                   =   dfPlayerProjStats.astype({"PTS": float, "REB": float, "AST": float, "ST": float, "BLK": float, \
                                                                      "TO": float, "FG%": float, "FT%": float, "3PTM": float})

    return  dfPlayerProjStats

In [7]:
def updateDraft(dfDraft, teamName, dfPlayerProjStats, playerIndex):
    player                          =   dfPlayerProjStats.iloc[playerIndex]
    
    # create new dataframe with draft_columns. add a single row with the player's stats and the team name
    dfPlayer                       =   pd.DataFrame(columns=draft_columns)
    dfPlayer.loc[0]                =   [teamName] + player.tolist()

    dfDraft                         =   pd.concat([dfDraft, dfPlayer], ignore_index=True)
    return dfDraft

In [8]:
def getTeamDraft(dfDraft, teamName):

    dfTeamDraft                =   dfDraft[dfDraft["Team"] == teamName]

    #print("Team Draft for ", teamName)
    #print(dfTeamDraft)

    return dfTeamDraft

In [9]:
def getTeamDraftStats(dfDraft, teamName):
    dfTeamDraft = getTeamDraft(dfDraft, teamName)

    # create a new dataframe with the relevant columns and sum the stats
    dfTeamDraftStats                    =   pd.DataFrame(columns= ["Team"] + relevantColumns)

    statsRow                           =   {'Team': teamName, \
                                            'Player': dfTeamDraft['Player'].sum(),
                                            'PTS': dfTeamDraft['PTS'].sum(), \
                                            'REB': dfTeamDraft['REB'].sum(), \
                                            'AST': dfTeamDraft['AST'].sum(), \
                                            'ST': dfTeamDraft['ST'].sum(), \
                                            'BLK': dfTeamDraft['BLK'].sum(), \
                                            'TO': dfTeamDraft['TO'].sum(), \
                                            '3PTM': dfTeamDraft['3PTM'].sum(), \
                                            'FG%': dfTeamDraft['FG%'].mean(), \
                                            'FT%': dfTeamDraft['FT%'].mean()}
    dfTeamDraftStats.loc[0]             =   statsRow
    
    #print("Team Draft Stats for: ", teamName)
    #print(dfTeamDraftStats)

    return dfTeamDraftStats


In [10]:
def getAllTeamsDraftStats(dfDraft):
    dfTeamStats =     pd.DataFrame(columns= ["Team"] + relevantColumns)
    
    for teamName in ROTO8_DRAFT_ORDER:
        dfTeamStats   =    pd.concat([dfTeamStats, getTeamDraftStats(dfDraft, teamName)], ignore_index=True)

    return dfTeamStats

In [11]:
def noramlizeTeamsDraftStats(dfAllTeamsDraftStats, teamPlayerCountDct, dfDraft):

    # find the maximum number of players selected by each team
    maxPlayerCount = max(teamPlayerCountDct.values())    

    # create a new dataframe with the relevant columns and sum the stats
    dfNormalizedTeamStats                    =   pd.DataFrame(columns= ["Team"] + relevantColumns)



    for i, teamName in enumerate(ROTO8_DRAFT_ORDER):

        # check if the team has selected any players
        if teamPlayerCountDct[teamName] == 0:
            continue

        # check if the team has selected the maximum number of players
        if teamPlayerCountDct[teamName] == maxPlayerCount:
            dfNormalizedTeamStats.loc[i] = dfAllTeamsDraftStats.loc[i]
            continue

        # Check if the team has selected less than the maximum number of players

        currTeamPlayerCount = teamPlayerCountDct[teamName]

        factor = maxPlayerCount/currTeamPlayerCount



        # find the stats for the team and divide by the number of players selected

        dfTeamDraftStats = getTeamDraftStats(dfDraft, teamName)

        #print("Team Draft Stats for: ", teamName)
        #print(dfTeamDraftStats)

        statsRow                           =  {'Team': teamName, \
                                            'Player': dfTeamDraftStats['Player'].values[0], \
                                            'PTS': float(int(dfTeamDraftStats['PTS'].values[0]*factor)), \
                                            'REB': float(int(dfTeamDraftStats['REB'].values[0]*factor)), \
                                            'AST': float(int(dfTeamDraftStats['AST'].values[0]*factor)), \
                                            'ST': float(int(dfTeamDraftStats['ST'].values[0]*factor)), \
                                            'BLK': float(int(dfTeamDraftStats['BLK'].values[0]*factor)), \
                                            'TO': float(int(dfTeamDraftStats['TO'].values[0]*factor)), \
                                            '3PTM': float(int(dfTeamDraftStats['3PTM'].values[0]*factor)), \
                                            'FG%': dfTeamDraftStats['FG%'].values[0], \
                                            'FT%': dfTeamDraftStats['FT%'].values[0]}
        



        dfNormalizedTeamStats.loc[i]             =   statsRow

    return dfNormalizedTeamStats

In [12]:
def getTeamRankings(dfTeamStats, myTeamName):
    # create ranking dataframe, sort by each category separately (PTS	REB	AST	ST	BLK	TO	FG%	FT%	3PTM) and assign ranks. 
    
    dfRankings = pd.DataFrame(columns=["Team"])
    # add the team names to the rankings dataframe as new rows
    dfRankings["Team"] = dfTeamStats["Team"]


    for category in relevantColumns:
        # print("Category: ", category)

        # keep only the category column and the Team column and sort by that category column
        dfCategory = dfTeamStats[["Team", category]].sort_values(by=category, ascending=True)

        # assign ranks
        dfCategory["Rank_" + category] = range(1, numTeams+1)

        # merge the ranks with the rankings dataframe
        dfRankings = pd.merge(dfRankings, dfCategory, on="Team")

    # calculate the total score for each team
    dfRankings["Total Score"] = dfRankings.filter(like="Rank").sum(axis=1)

    # sort by total score
    dfRankings = dfRankings.sort_values(by="Total Score", ascending=False)

    # find the row index of my team
    myTeamData = dfRankings[dfRankings["Team"] == myTeamName]

    # move my team to the top
    dfRankings = pd.concat([myTeamData, dfRankings[dfRankings["Team"] != myTeamName]])
    
    

    return dfRankings

In [13]:
def evaluateSelection(dfDraft, dfPlayerProjStats, myTeamName, playerIndex):
    # update the draft dataframe with the new player
    dfDraft = updateDraft(dfDraft, myTeamName, dfPlayerProjStats, playerIndex)
    # calculate the total score for each team
    dfTeamStats = getAllTeamsDraftStats(dfDraft)
    dfRankings = getTeamRankings(dfTeamStats, myTeamName)   
    return dfRankings

In [14]:
def evaluateSelections(dfDraft, dfPlayerProjStats, myTeamName, playerIndexLst):   
    dctSelectionsRankings = {}       
    for playerIndex in playerIndexLst:
        dfRankings = evaluateSelection(dfDraft, dfPlayerProjStats, myTeamName, playerIndex)
        #print("Player Index: ", playerIndex)
        #print(dfRankings)
        #print("\n\n")

        dctSelectionsRankings[playerIndex] = dfRankings

    return dctSelectionsRankings

In [15]:
def getMyTeamRankings(playerIndexLst, dfDraft, dfPlayerProjStats, myTeamName, dfRankings):
    dctSelectionsRankings                           =   evaluateSelections(dfDraft, dfPlayerProjStats, myTeamName, playerIndexLst)

    dfMyTeamRankings                                =   pd.DataFrame(columns=dctSelectionsRankings[playerIndexLst[0]].columns)
                                        
    # iterate over the dictionary and print the rankings for each player selection
    for playerIndex in playerIndexLst:
        #print("Player Index: ", playerIndex)
        currSelectionRank                 =   dctSelectionsRankings[playerIndex]

        myTeamData = currSelectionRank[currSelectionRank["Team"] == myTeamName]

        myTeamData["Player"] = dfPlayerProjStats.iloc[playerIndex]["Player"]
        
        dfMyTeamRankings = pd.concat([dfMyTeamRankings, myTeamData])

    # sort by total score
    dfMyTeamRankings = dfMyTeamRankings.sort_values(by="Total Score", ascending=False)

    # get my current team ranking from dfRankings
    myTeamRank = dfRankings[dfRankings["Team"] == myTeamName]


    dfMyTeamRankings = pd.concat([myTeamRank ,dfMyTeamRankings])

    dfMyTeamRankings

    return dfMyTeamRankings


# Draft

In [16]:
dfPlayerProjStats                       =   loadPlayerProjStats(playerProjStatsFilepath)
dfPlayerProjStats.head()

loadPlayerProjStats
Loading player stats from:  ../data/proj_24_25_150_players.csv


,Player,PTS,REB,AST,ST,BLK,TO,3PTM,FG%,FT%
0,V. Wembanyama,1849.0,855.0,358.0,95.0,305.0,282.0,173.0,0.476,0.809
1,N. Jokić,1970.0,916.0,715.0,94.0,63.0,293.0,83.0,0.604,0.820
2,L. Doncic,2232.0,626.0,574.0,82.0,40.0,321.0,269.0,0.486,0.761
3,S. Gilgeous-Alexander,2180.0,383.0,437.0,126.0,62.0,183.0,93.0,0.521,0.884
4,T. Haliburton,1410.0,280.0,745.0,102.0,42.0,181.0,205.0,0.468,0.865


In [17]:
dfDraft                             =   pd.DataFrame(columns=draft_columns)   

# create a list of number of players selected per team. initialize to 0 for all teams
teamPlayerCountDct                  = {}

for team in ROTO8_DRAFT_ORDER:
    teamPlayerCountDct[team]        = 0

stopIndex = 51 # numPlayersPerTeam*numTeams

index = 0
for choise in range(0, numPlayersPerTeam):    
    for i, team in enumerate(ROTO8_DRAFT_ORDER):
        if index >= stopIndex:
            break

        dfDraft                         =   updateDraft(dfDraft, team, dfPlayerProjStats, index)
        teamPlayerCountDct[team]        +=  1
        index                           +=  1

dfDraft# , teamPlayerCountDct

,Team,Player,PTS,REB,AST,ST,BLK,TO,3PTM,FG%,FT%
0,Team 1,V. Wembanyama,1849.0,855.0,358.0,95.0,305.0,282.0,173.0,0.476,0.809
1,Team 2,N. Jokić,1970.0,916.0,715.0,94.0,63.0,293.0,83.0,0.604,0.820
2,Team 3,L. Doncic,2232.0,626.0,574.0,82.0,40.0,321.0,269.0,0.486,0.761
3,Team 4,S. Gilgeous-Alexander,2180.0,383.0,437.0,126.0,62.0,183.0,93.0,0.521,0.884
4,Team 5,T. Haliburton,1410.0,280.0,745.0,102.0,42.0,181.0,205.0,0.468,0.865
5,Team 6,G. Antetokounmpo,2049.0,796.0,419.0,79.0,82.0,236.0,37.0,0.590,0.666
6,Team 7,J. Embiid,1956.0,631.0,303.0,69.0,99.0,217.0,76.0,0.528,0.868
7,Team 8,J. Tatum,1976.0,566.0,310.0,70.0,46.0,202.0,226.0,0.469,0.845
8,Team 1,A. Davis,1708.0,852.0,224.0,83.0,163.0,142.0,29.0,0.551,0.801
9,Team 2,K. Durant,1722.0,426.0,320.0,56.0,65.0,225.0,142.0,0.520,0.853


In [18]:
dfAllTeamsDraftStats                                 =   getAllTeamsDraftStats(dfDraft)
dfAllTeamsDraftStats

,Team,Player,PTS,REB,AST,ST,BLK,TO,3PTM,FG%,FT%
0,Team 1,V. WembanyamaA. DavisD. SabonisD. FoxC. Cunnin...,11053.0,3963.0,2525.0,565.0,767.0,1356.0,863.0,0.502571,0.800286
1,Team 2,N. JokićK. DurantJ. HardenL. JamesJ. JohnsonJ....,11161.0,3466.0,3354.0,518.0,326.0,1482.0,812.0,0.507571,0.811857
2,Team 3,L. DoncicT. YoungL. BallL. MarkkanenK. TownsJ....,11191.0,3465.0,2784.0,522.0,296.0,1443.0,1186.0,0.494429,0.837714
3,Team 4,S. Gilgeous-AlexanderS. CurryS. BarnesT. Maxey...,9994.0,2602.0,2301.0,515.0,373.0,1041.0,962.0,0.485667,0.856333
4,Team 5,T. HaliburtonD. MitchellJ. BrunsonD. MurrayP. ...,10108.0,2284.0,2746.0,537.0,189.0,1110.0,1037.0,0.471667,0.827500
5,Team 6,G. AntetokounmpoK. IrvingC. HolmgrenK. Leonard...,9128.0,2737.0,2066.0,477.0,406.0,945.0,756.0,0.503667,0.829667
6,Team 7,J. EmbiidD. BookerD. LillardJ. Jackson Jr.D. B...,9729.0,2815.0,1827.0,416.0,401.0,942.0,797.0,0.512167,0.839500
7,Team 8,J. TatumA. EdwardsP. GeorgeA. SengunE. MobleyJ...,10149.0,3169.0,1904.0,505.0,328.0,1284.0,891.0,0.499667,0.794167


In [26]:
#test
if False:
    teamPlayerCountDct['Team 5'] = 12
    teamPlayerCountDct['Team 6'] = 12
    teamPlayerCountDct['Team 7'] = 12
    teamPlayerCountDct['Team 8'] = 12

teamPlayerCountDct

{'Team 1': 7,
 'Team 2': 7,
 'Team 3': 7,
 'Team 4': 6,
 'Team 5': 6,
 'Team 6': 6,
 'Team 7': 6,
 'Team 8': 6}

In [27]:
dfNormAllTeamsDraftStats                            =   noramlizeTeamsDraftStats(dfAllTeamsDraftStats, teamPlayerCountDct, dfDraft)
dfNormAllTeamsDraftStats

,Team,Player,PTS,REB,AST,ST,BLK,TO,3PTM,FG%,FT%
0,Team 1,V. WembanyamaA. DavisD. SabonisD. FoxC. Cunnin...,11053.0,3963.0,2525.0,565.0,767.0,1356.0,863.0,0.502571,0.800286
1,Team 2,N. JokićK. DurantJ. HardenL. JamesJ. JohnsonJ....,11161.0,3466.0,3354.0,518.0,326.0,1482.0,812.0,0.507571,0.811857
2,Team 3,L. DoncicT. YoungL. BallL. MarkkanenK. TownsJ....,11191.0,3465.0,2784.0,522.0,296.0,1443.0,1186.0,0.494429,0.837714
3,Team 4,S. Gilgeous-AlexanderS. CurryS. BarnesT. Maxey...,11659.0,3035.0,2684.0,600.0,435.0,1214.0,1122.0,0.485667,0.856333
4,Team 5,T. HaliburtonD. MitchellJ. BrunsonD. MurrayP. ...,11792.0,2664.0,3203.0,626.0,220.0,1295.0,1209.0,0.471667,0.8275
5,Team 6,G. AntetokounmpoK. IrvingC. HolmgrenK. Leonard...,10649.0,3193.0,2410.0,556.0,473.0,1102.0,882.0,0.503667,0.829667
6,Team 7,J. EmbiidD. BookerD. LillardJ. Jackson Jr.D. B...,11350.0,3284.0,2131.0,485.0,467.0,1099.0,929.0,0.512167,0.8395
7,Team 8,J. TatumA. EdwardsP. GeorgeA. SengunE. MobleyJ...,11840.0,3697.0,2221.0,589.0,382.0,1498.0,1039.0,0.499667,0.794167


In [28]:
dfAllTeamsDraftStats

,Team,Player,PTS,REB,AST,ST,BLK,TO,3PTM,FG%,FT%
0,Team 1,V. WembanyamaA. DavisD. SabonisD. FoxC. Cunnin...,11053.0,3963.0,2525.0,565.0,767.0,1356.0,863.0,0.502571,0.800286
1,Team 2,N. JokićK. DurantJ. HardenL. JamesJ. JohnsonJ....,11161.0,3466.0,3354.0,518.0,326.0,1482.0,812.0,0.507571,0.811857
2,Team 3,L. DoncicT. YoungL. BallL. MarkkanenK. TownsJ....,11191.0,3465.0,2784.0,522.0,296.0,1443.0,1186.0,0.494429,0.837714
3,Team 4,S. Gilgeous-AlexanderS. CurryS. BarnesT. Maxey...,9994.0,2602.0,2301.0,515.0,373.0,1041.0,962.0,0.485667,0.856333
4,Team 5,T. HaliburtonD. MitchellJ. BrunsonD. MurrayP. ...,10108.0,2284.0,2746.0,537.0,189.0,1110.0,1037.0,0.471667,0.827500
5,Team 6,G. AntetokounmpoK. IrvingC. HolmgrenK. Leonard...,9128.0,2737.0,2066.0,477.0,406.0,945.0,756.0,0.503667,0.829667
6,Team 7,J. EmbiidD. BookerD. LillardJ. Jackson Jr.D. B...,9729.0,2815.0,1827.0,416.0,401.0,942.0,797.0,0.512167,0.839500
7,Team 8,J. TatumA. EdwardsP. GeorgeA. SengunE. MobleyJ...,10149.0,3169.0,1904.0,505.0,328.0,1284.0,891.0,0.499667,0.794167


In [22]:
dfRankings = getTeamRankings(dfAllTeamsDraftStats,myTeamName)
dfRankings

,Team,Player,Rank_Player,PTS,Rank_PTS,REB,Rank_REB,AST,Rank_AST,ST,...,Rank_BLK,TO,Rank_TO,3PTM,Rank_3PTM,FG%,Rank_FG%,FT%,Rank_FT%,Total Score
6,Team 7,J. EmbiidD. BookerD. LillardJ. Jackson Jr.D. B...,2,9729.0,2,2815.0,4,1827.0,1,416.0,...,6,942.0,1,797.0,2,0.512167,8,0.839500,7,34
0,Team 1,V. WembanyamaA. DavisD. SabonisD. FoxC. Cunnin...,8,11053.0,6,3963.0,8,2525.0,5,565.0,...,8,1356.0,6,863.0,4,0.502571,5,0.800286,2,60
2,Team 3,L. DoncicT. YoungL. BallL. MarkkanenK. TownsJ....,4,11191.0,8,3465.0,6,2784.0,7,522.0,...,2,1443.0,7,1186.0,8,0.494429,3,0.837714,6,57
1,Team 2,N. JokićK. DurantJ. HardenL. JamesJ. JohnsonJ....,5,11161.0,7,3466.0,7,3354.0,8,518.0,...,3,1482.0,8,812.0,3,0.507571,7,0.811857,3,56
3,Team 4,S. Gilgeous-AlexanderS. CurryS. BarnesT. Maxey...,6,9994.0,3,2602.0,2,2301.0,4,515.0,...,5,1041.0,3,962.0,6,0.485667,2,0.856333,8,43
4,Team 5,T. HaliburtonD. MitchellJ. BrunsonD. MurrayP. ...,7,10108.0,4,2284.0,1,2746.0,6,537.0,...,1,1110.0,4,1037.0,7,0.471667,1,0.827500,4,42
7,Team 8,J. TatumA. EdwardsP. GeorgeA. SengunE. MobleyJ...,3,10149.0,5,3169.0,5,1904.0,2,505.0,...,4,1284.0,5,891.0,5,0.499667,4,0.794167,1,37
5,Team 6,G. AntetokounmpoK. IrvingC. HolmgrenK. Leonard...,1,9128.0,1,2737.0,3,2066.0,3,477.0,...,7,945.0,2,756.0,1,0.503667,6,0.829667,5,31


In [29]:
dfNormRankings = getTeamRankings(dfNormAllTeamsDraftStats,myTeamName)
dfNormRankings

,Team,Player,Rank_Player,PTS,Rank_PTS,REB,Rank_REB,AST,Rank_AST,ST,...,Rank_BLK,TO,Rank_TO,3PTM,Rank_3PTM,FG%,Rank_FG%,FT%,Rank_FT%,Total Score
6,Team 7,J. EmbiidD. BookerD. LillardJ. Jackson Jr.D. B...,2,11350.0,5,3284.0,4,2131.0,1,485.0,...,6,1099.0,1,929.0,4,0.512167,8,0.8395,7,39
3,Team 4,S. Gilgeous-AlexanderS. CurryS. BarnesT. Maxey...,6,11659.0,6,3035.0,2,2684.0,5,600.0,...,5,1214.0,3,1122.0,6,0.485667,2,0.856333,8,50
0,Team 1,V. WembanyamaA. DavisD. SabonisD. FoxC. Cunnin...,8,11053.0,2,3963.0,8,2525.0,4,565.0,...,8,1356.0,5,863.0,2,0.502571,5,0.800286,2,49
4,Team 5,T. HaliburtonD. MitchellJ. BrunsonD. MurrayP. ...,7,11792.0,7,2664.0,1,3203.0,7,626.0,...,1,1295.0,4,1209.0,8,0.471667,1,0.8275,4,48
7,Team 8,J. TatumA. EdwardsP. GeorgeA. SengunE. MobleyJ...,3,11840.0,8,3697.0,7,2221.0,2,589.0,...,4,1498.0,8,1039.0,5,0.499667,4,0.794167,1,48
2,Team 3,L. DoncicT. YoungL. BallL. MarkkanenK. TownsJ....,4,11191.0,4,3465.0,5,2784.0,6,522.0,...,2,1443.0,6,1186.0,7,0.494429,3,0.837714,6,46
1,Team 2,N. JokićK. DurantJ. HardenL. JamesJ. JohnsonJ....,5,11161.0,3,3466.0,6,3354.0,8,518.0,...,3,1482.0,7,812.0,1,0.507571,7,0.811857,3,45
5,Team 6,G. AntetokounmpoK. IrvingC. HolmgrenK. Leonard...,1,10649.0,1,3193.0,3,2410.0,3,556.0,...,7,1102.0,2,882.0,3,0.503667,6,0.829667,5,35


In [24]:
dfNormAllTeamsDraftStats

,Team,Player,PTS,REB,AST,ST,BLK,TO,3PTM,FG%,FT%
0,Team 1,V. WembanyamaA. DavisD. SabonisD. FoxC. Cunnin...,11053.0,3963.0,2525.0,565.0,767.0,1356.0,863.0,0.502571,0.800286
1,Team 2,N. JokićK. DurantJ. HardenL. JamesJ. JohnsonJ....,11161.0,3466.0,3354.0,518.0,326.0,1482.0,812.0,0.507571,0.811857
2,Team 3,L. DoncicT. YoungL. BallL. MarkkanenK. TownsJ....,11191.0,3465.0,2784.0,522.0,296.0,1443.0,1186.0,0.494429,0.837714
3,Team 4,S. Gilgeous-AlexanderS. CurryS. BarnesT. Maxey...,11659.0,3035.0,2684.0,600.0,435.0,1214.0,1122.0,0.485667,0.856333
4,Team 5,T. HaliburtonD. MitchellJ. BrunsonD. MurrayP. ...,11792.0,2664.0,3203.0,626.0,220.0,1295.0,1209.0,0.471667,0.8275
5,Team 6,G. AntetokounmpoK. IrvingC. HolmgrenK. Leonard...,10649.0,3193.0,2410.0,556.0,473.0,1102.0,882.0,0.503667,0.829667
6,Team 7,J. EmbiidD. BookerD. LillardJ. Jackson Jr.D. B...,11350.0,3284.0,2131.0,485.0,467.0,1099.0,929.0,0.512167,0.8395
7,Team 8,J. TatumA. EdwardsP. GeorgeA. SengunE. MobleyJ...,11840.0,3697.0,2221.0,589.0,382.0,1498.0,1039.0,0.499667,0.794167


In [25]:
playerIndexLst                                  =   [10, 20, 30]
dfMyTeamRankings                                =   getMyTeamRankings(playerIndexLst, dfDraft, dfPlayerProjStats, myTeamName, dfRankings)
dfMyTeamRankings

,Team,Player,Rank_Player,PTS,Rank_PTS,REB,Rank_REB,AST,Rank_AST,ST,...,Rank_BLK,TO,Rank_TO,3PTM,Rank_3PTM,FG%,Rank_FG%,FT%,Rank_FT%,Total Score
6,Team 7,J. EmbiidD. BookerD. LillardJ. Jackson Jr.D. B...,2,9729.0,2,2815.0,4,1827.0,1,416.0,...,6,942.0,1,797.0,2,0.512167,8,0.839500,7,34
6,Team 7,T. Young,2,11725.0,8,3043.0,4,2565.0,5,504.0,...,7,1254.0,4,1034.0,6,0.499857,5,0.844286,7,50
6,Team 7,J. Brunson,2,11656.0,8,3089.0,4,2343.0,4,486.0,...,6,1121.0,4,981.0,6,0.507286,7,0.839714,7,50
6,Team 7,J. Jackson Jr.,2,11176.0,7,3265.0,5,1953.0,2,499.0,...,7,1082.0,3,920.0,5,0.509000,8,0.834286,6,47
